In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Point
from PIL import Image, ImageDraw
import glob
import os

In [2]:
FOLDER = "/Users/damoncrockett/Desktop/stpete/viz/hists-tsne/"

In [3]:
df = pd.read_csv("./hists_imgs.csv")

In [4]:
thumb_side = 128

In [6]:
d = pd.read_csv("./tags_counts_rank.csv")

In [8]:
del d['Unnamed: 0']

In [11]:
d.set_index("tag",inplace=True)
df = df.join(d,on='tag')

In [15]:
df = df[df.local_path.notnull()]

In [17]:
df.reset_index(drop=True,inplace=True)

In [33]:
col = 'count'

In [34]:
tmp = df

In [35]:
tmp.sort(col,inplace=True,ascending=False)
tmp.reset_index(drop=True,inplace=True)
n = len(tmp)

side = int(round(np.sqrt(n))) + 10
canvas = Image.new('RGB',(side * thumb_side, side * thumb_side),(255,255,255))
x,y = range(side) * side, np.repeat(range(side),side)
grid_list = pd.DataFrame(x,columns=['x'])
grid_list['y'] = y

point = []
l = len(grid_list.index)
for i in range(l):
    point.append(Point(grid_list.x.loc[i],grid_list.y.loc[i]))
grid_list['point'] = point
open_grid = list(grid_list.point)

exemplar = Point(int(round(side/2)),int(round(side/2)))
open_grid.remove(exemplar)

for i in range(n):
    im = Image.open(tmp.local_path.loc[i])
    im.thumbnail((thumb_side,thumb_side),Image.ANTIALIAS)
    draw = ImageDraw.Draw(im)
    draw.text((5,5),str(tmp[col].loc[i]),fill=(155,155,155))
    draw.text((5,115),str(tmp.tag.loc[i]),fill=(155,155,155))
    
    closest_open = min(open_grid,key=lambda x: exemplar.distance(x))
    x = int(closest_open.x) * thumb_side
    y = int(closest_open.y) * thumb_side
    canvas.paste(im,(x,y))
    open_grid.remove(closest_open)

canvas.save(FOLDER+col+".png")